Anévrisme

# Introduction

## Paquets nécessaires

ipympl

In [1]:
%matplotlib widget
import copy
from helpers import *
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive
from IPython.display import display, clear_output

# Lecture du DICOMDIR et traitement des images

Dans un premier temps, nous avons recherché un outil en Python capable de lire des fichiers DICOM et d'en extraire les informations nécessaires. Le paquet `pydicom` permet ceci.

<!-- Description de l'installation -->

Une fois ce paquet installé, nous pouvons lire notre jeu de données. Celui-ci contient des informations à plusieurs niveaux : pour chaque patient, il peut y avoir une ou plusieurs expérience(s) contenant chacune une ou plusieurs série(s) d'images. Dans notre cas, nous n'avons qu'une seule série. Nous récupérons les chemins de ces images puis les lisons avec la fonction `dcmread` du paquet `pydicom`.

<!-- On a remarqué que sur les 500 et quelques images qu'on a, il y a une répétition toutes les 48 images -->

Ces images sont en niveaux de gris sur 16 bits.

In [2]:
dataset_path = "ImgTP/Ge1CaroG/MR_3DPCA"
ds = load_dataset(dataset_path)

# 0018,1090  Cardiac Number of Images: 12
nb_image_sets = 12

# Select the first image set (must be < 12)
starting_image_set = 0

In [3]:
data = {}

# Iterate through the PATIENT records
for patient in ds.patient_records:
    # Find all the STUDY records for the patient
    studies = [ii for ii in patient.children if ii.DirectoryRecordType == "STUDY"]
    
    data[patient.PatientName] = {}

    for study in studies:
        # Find all the SERIES records in the study
        all_series = [ii for ii in study.children if ii.DirectoryRecordType == "SERIES"]
        
        data[patient.PatientName][study.StudyID] = {}

        for series in all_series:
            data[patient.PatientName][study.StudyID][series.SeriesNumber] = {}
            
            # Find all the IMAGE records in the series
            images = [ii for ii in series.children if ii.DirectoryRecordType == "IMAGE"]
            
            # Get the absolute file path to each instance
            # Each IMAGE contains a relative file path to the root directory
            elems = [ii["ReferencedFileID"] for ii in images]
            # Make sure the relative file path is always a list of str
            paths = [[ee.value] if ee.VM == 1 else ee.value for ee in elems]
            paths = [Path(*p) for p in paths]

            images = []

            i = 0
            
            total_images = len(paths)
            max_images = int(total_images / nb_image_sets)
            
            # List the instance file paths
            for idx in range(starting_image_set * max_images, (starting_image_set + 1) * max_images):
                p = paths[idx]
                img = dcmread(Path(dataset_path).joinpath(p))
                images.append((img, p))
                    
            data[patient.PatientName][study.StudyID][series.SeriesNumber]['images'] = images

## Suppression du bruit

La suppression du bruit peut s'effectuer par l'utilisation de différents filtres. Parmi ceux-ci on retrouve par exemple le filtre bilatéral, le débruitage par patchs (Non-Local Means) ou encore le filtre médian.

In [4]:
from skimage.restoration import denoise_nl_means, estimate_sigma, denoise_bilateral

def apply_non_local_means(img, kernel=5, window_search=13):
    # Retain original data type
    orig_dtype = img.pixel_array.dtype

    # Convert from [0; max] to [0; 1] as it is required by denoise_nl_means
    upper_bound = np.max(img.pixel_array)
    img_as_float = img.pixel_array / upper_bound

    sigma_est = np.mean(estimate_sigma(img_as_float, multichannel=False))

    new_img = denoise_nl_means(img_as_float, h=sigma_est, fast_mode=True, patch_size=kernel,
                               patch_distance=window_search)

    # Convert back to [0; max]
    new_img *= upper_bound

    return new_img.astype(orig_dtype)


def apply_bilateral_filtering(img, d=15, sigmacolor=75, sigmacoordinate=75):
    # Retain original data type
    orig_dtype = img.pixel_array.dtype

    # Convert from [0; max] to [0; 1] as it is required by denoise_nl_means
    upper_bound = np.max(img.pixel_array)
    img_as_float = img.pixel_array / upper_bound

    new_img = denoise_bilateral(img_as_float, win_size=d, sigma_color=sigmacolor, sigma_spatial=sigmacoordinate)

    # Convert back to [0; max]
    new_img *= upper_bound

    return new_img.astype(orig_dtype)

In [5]:
for patientName, patientData in data.items():
    for studyId, studyData in patientData.items():
        for seriesId, seriesData in studyData.items():
            images = seriesData['images']
            denoised_images = []
            
            for image, p in images:
                median = apply_simple_denoise(image, kernel_size=3)

                denoised_images.append(median)
                
            data[patientName][studyId][seriesId]['denoised_images'] = denoised_images

## Segmentation

In [6]:
for patientName, patientData in data.items():
    for studyId, studyData in patientData.items():
        for seriesId, seriesData in studyData.items():
            denoised_images = data[patientName][studyId][seriesId]['denoised_images']
            threshold_images = []
            random_walker_images = []
            fills = {}
    
            for median_image in denoised_images:
                thresh = apply_threshold(median_image)
                random_walker = apply_random_walker(median_image)

                for tol in np.linspace(0.2, 0.4, 12):
                    str_tol = "{:.2f}".format(tol)

                    if str_tol not in fills:
                        fills[str_tol] = []

                    fill = apply_flood_fill(median_image, (76, 69), tol)
                    fills[str_tol].append(fill)

                threshold_images.append(thresh)
                random_walker_images.append(random_walker)

# Sélection des méthodes de segmentation (WIP)

In [16]:
segmentation_methods = [
    'Flood',
    'Random Walker'
]

segmentation_method_widget = widgets.Dropdown(
    options=segmentation_methods,
    value='Flood',
    description='Segmentation method:',
)

In [44]:
# %%html
# <style>
# div.jupyter-widgets-view:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) {display: initial;}
# </style>

# Préparation du plot et des widgets 

In [17]:
image_sets = [
    {
        'images': [image.pixel_array for image, _ in images],
        'title': 'Original',
        'shown': True
    },
    {
        'images': denoised_images,
        'title': 'Median Filter',
        'shown': False
    },
    {
        'images': threshold_images,
        'title': 'Threshold',
        'shown': False
    },
    {
        'images': random_walker_images,
        'title': 'Random Walker',
        'shown': False
    }
]

for tol in fills:
    image_sets.append({
        'images': fills[tol],
        'title': 'Flood Fill Tol {}'.format(tol),
        'shown': False
    })

nb_image_sets = len(image_sets)

In [18]:
output = widgets.Output()

fig = None

with output:
    plt.close(fig)
    fig = plt.figure()

# Hide figure header
fig.canvas.header_visible = False
ls = []
current_image_slider = 0
shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
zoom = 1
output_h, output_w = 0, 0

# Executed on image slider change
def update_images(change):
    global shown_image_sets, ls
    current_image_slider = change.new

    for idx, image in enumerate(ls):
        image.set_data(shown_image_sets[idx]['images'][change.new])

# Executed on zoom slider change
def update_zoom(change):
    global zoom, fig
    zoom = change.new
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)

# Executed on image sets selection
def update_image_sets(change):
    global image_sets, ls, fig
    ls.clear()

    for image_set in image_sets: image_set['shown'] = False
    
    for idx in change.owner.index:
        image_sets[idx]['shown'] = True
    
    plot_images(fig, image_sets)

def plot_images(fig, image_sets):
    global shown_image_sets, output_w, output_h
    shown_image_sets = [image_set for image_set in image_sets if image_set['shown']]
    nb_shown_image_sets = len(shown_image_sets)
    
    ncols = int(np.ceil(np.sqrt(nb_shown_image_sets)))
    nrows = int(np.ceil(nb_shown_image_sets / float(ncols)))
    height, width = shown_image_sets[0]['images'][0].shape
    output_h, output_w = (height * nrows) + 100, (width * ncols) + 100
    
    # Clear previous figure
    fig.clf()
    # Set figure size based on the number of images
    fig.set_size_inches(output_w * zoom / 100, output_h * zoom / 100, forward=True)
    fig.set_dpi(100)
    
    for idx in range(nb_shown_image_sets):
        ax = fig.add_subplot(nrows, ncols, idx + 1)
        image = ax.imshow(shown_image_sets[idx]['images'][current_image_slider], cmap=plt.cm.gray)
        ls.append(image)
        
        # Set title
        ax.title.set_text(shown_image_sets[idx]['title'])
        
        # Hide grid and x, y ticks
        ax.grid(False)
        plt.xticks([])
        plt.yticks([])
    
image_slider = widgets.IntSlider(
    value=0, 
    min=0, max=47, step=1,
    continuous_update=True,
    description='Image',
    layout=widgets.Layout(width='99%')
)

zoom_slider = widgets.FloatSlider(
    value=1,
    min=1, max=3, step=0.1,
    continuous_update=False,
    description='Zoom',
    layout=widgets.Layout(width='99%')
)

image_set_selection = widgets.SelectMultiple(
    # Add all image sets as options 
    options=[image_set['title'] for image_set in image_sets],
    # Select by default the ones that are set as 'shown'
    value=[image_set['title'] for image_set in shown_image_sets],
    disabled=False,
    layout=widgets.Layout(width='99%', align_items='stretch')
)

image_slider.observe(update_images, 'value')
zoom_slider.observe(update_zoom, 'value')
image_set_selection.observe(update_image_sets, 'value')

plt.subplots_adjust(wspace=0.5, hspace=0.5)

# Affichage des images

In [19]:
plot_images(fig, shown_image_sets)

widgets.VBox([image_set_selection, output, image_slider, zoom_slider])